In [ ]:
import sys
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import keras
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.plotting import scatter_matrix

In [ ]:
#prepare my ,achine for direct import dataset

#from google.colab import drive
#drive.mount("/content/gdrive")




In [43]:
#creating a uploader and upload my dataset

from google.colab import files
uploaded = files.upload()

#url = 'https://drive.google.com/file/d/11JSeNoUgS_thEtI7LaaKEFqcUZdSOP0c/view?usp=drive_link'

Saving heart.csv to heart (1).csv


In [44]:
#read uploaded csv file
cleveland = pd.read_csv('heart.csv')

In [ ]:
print(cleveland)

In [45]:
# print the shape of the DataFrame, so we can see how many examples we have
print( 'Shape of DataFrame: {}'.format(cleveland.shape))

Shape of DataFrame: (1025, 14)


# data preprocessing

In [ ]:
# remove missing data (indicated with a "?")
data = cleveland[~cleveland.isin(['?'])]
data.loc[0:]

# drop rows with NaN values from DataFrame
data = data.dropna(axis=0)
data.loc[0:]

In [ ]:
# print the shape and data type of the dataframe
print(data.shape)
print(data.dtypes)

In [ ]:
# transform data to numeric to enable further analysis
data = data.apply(pd.to_numeric)
data.dtypes

data.describe()

# creating training and test dataset

In [50]:
X = np.array(data.drop(['target'], 1))
y = np.array(data['target'])

X[0]

<ipython-input-50-fa4d3e1bbf96>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(data.drop(['target'], 1))


array([ 52.,   1.,   0., 125., 212.,   0.,   1., 168.,   0.,   1.,   2.,
         2.,   3.])

In [51]:
mean = X.mean(axis=0)
X -= mean
std = X.std(axis=0)
X /= std

X[0]

array([-0.26843658,  0.66150409, -0.91575542, -0.37763552, -0.65933209,
       -0.41887792,  0.89125488,  0.82132052, -0.71228712, -0.06088839,
        0.99543334,  1.20922066,  1.08985168])

In [52]:
# create X and Y datasets for training
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y, random_state=42, test_size = 0.2)

In [53]:
# convert the data to categorical labels
from keras.utils import to_categorical



In [54]:
Y_train = to_categorical(y_train, num_classes=None)
Y_test = to_categorical(y_test, num_classes=None)
print (Y_train.shape)
print (Y_train[:10])

(820, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [ ]:
X_train[0]

In [ ]:
y_train[819]

In [57]:
from tensorflow.keras.models import Model

In [58]:
#try to get accuracy but no solution cz of noisse data
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
accuraccy = accuracy_score(y_test,y_pred)
print ("acc:", accuraccy)

acc: 0.9853658536585366


# Building and training Neural network


In [59]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import regularizers


In [77]:

# define a function to build the keras model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, input_dim=13, kernel_initializer='normal',  activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(32, kernel_initializer='normal',  activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))

    # compile model
    adam = Adam
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

model = create_model()

print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 64)                896       
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_31 (Dense)            (None, 32)                2080      
                                                                 
 dropout_21 (Dropout)        (None, 32)                0         
                                                                 
 dense_32 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3009 (11.75 KB)
Trainable params: 3009 (11.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# fit the model to the training data
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
#history= model.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=50, batch_size=20)

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline
# Model accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# Model Losss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

# Improving Results - A Binary Classification Problem

In [80]:
# convert into binary classification problem - heart disease or no heart disease
Y_train_binary = y_train.copy()
Y_test_binary = y_test.copy()

Y_train_binary[Y_train_binary > 0] = 1
Y_test_binary[Y_test_binary > 0] = 1

print(Y_train_binary[:20])
print(Y_test_binary[:20])

[0 0 0 1 1 0 0 1 1 0 1 1 0 0 0 1 0 1 1 0]
[0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 1]


In [ ]:
# define a new keras model for binary classification
def create_binary_model():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=13, kernel_initializer='normal',  activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(8, kernel_initializer='normal',  activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    adam = Adam
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

binary_model = create_binary_model()

print(binary_model.summary())

In [88]:
# fit the model to the training data
#history=model.fit(X_train, Y_train, validation_data=(X_test, Y_test),epochs=50, batch_size=20)
history = binary_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
26/26 [==============================] - 18s 10ms/step - loss: 0.6780 - accuracy: 0.5476 - val_loss: 0.6558 - val_accuracy: 0.6390
Epoch 2/20
26/26 [==============================] - 0s 3ms/step - loss: 0.6373 - accuracy: 0.6817 - val_loss: 0.6057 - val_accuracy: 0.7463
Epoch 3/20
26/26 [==============================] - 0s 4ms/step - loss: 0.5923 - accuracy: 0.7732 - val_loss: 0.5603 - val_accuracy: 0.7707
Epoch 4/20
26/26 [==============================] - 0s 4ms/step - loss: 0.5531 - accuracy: 0.8110 - val_loss: 0.5285 - val_accuracy: 0.8098
Epoch 5/20
26/26 [==============================] - 0s 4ms/step - loss: 0.5292 - accuracy: 0.8329 - val_loss: 0.5062 - val_accuracy: 0.8146
Epoch 6/20
26/26 [==============================] - 0s 4ms/step - loss: 0.5052 - accuracy: 0.8488 - val_loss: 0.4855 - val_accuracy: 0.8293
Epoch 7/20
26/26 [==============================] - 0s 3ms/step - loss: 0.4797 - accuracy: 0.8573 - val_loss: 0.4655 - val_accuracy: 0.8244
Epoch 8/20
26/26 [

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline
# Model accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# Model Losss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()

# result and matrix

In [93]:
# generate classification report using predictions for categorical model
from sklearn.metrics import classification_report, accuracy_score

categorical_pred = np.argmax(model.predict(X_test), axis=1)

print('Results for Categorical Model')
print(accuracy_score(y_test, categorical_pred))
print(classification_report(y_test, categorical_pred))

7/7 [==============================] - 0s 3ms/step
Results for Categorical Model
0.4878048780487805
              precision    recall  f1-score   support

           0       0.49      1.00      0.66       100
           1       0.00      0.00      0.00       105

    accuracy                           0.49       205
   macro avg       0.24      0.50      0.33       205
weighted avg       0.24      0.49      0.32       205



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [94]:
# generate classification report using predictions for binary model
from sklearn.metrics import classification_report, accuracy_score

binary_pred = np.round(binary_model.predict(X_test)).astype(int)

print('Results for Binary Model')
print(accuracy_score(Y_test_binary, binary_pred))
print(classification_report(Y_test_binary, binary_pred))

7/7 [==============================] - 0s 2ms/step
Results for Binary Model
0.8731707317073171
              precision    recall  f1-score   support

           0       0.90      0.83      0.86       100
           1       0.85      0.91      0.88       105

    accuracy                           0.87       205
   macro avg       0.88      0.87      0.87       205
weighted avg       0.88      0.87      0.87       205

